In [1]:
from makammusicbrainz.audiometadata import AudioMetadata
from ahenkidentifier.ahenkidentifier import AhenkIdentifier
import json
import os


First, we crawl the metadata from MusicBrainz using [makammusicbrainz](https://github.com/sertansenturk/makammusicbrainz).

In [2]:
anno_dict = json.load(open('../annotations.json'))
mbids = anno_dict.keys()
audio_meta_crawler = AudioMetadata()

for ii, mbid in enumerate(mbids):
    save_file = os.path.join('../metadata/%s.json' % mbid)
    if not os.path.exists(save_file):
        print("Crawling %s" %mbid)
        meta = audio_meta_crawler.from_musicbrainz(mbid)
        json.dump(meta, open(save_file, 'w'), indent=4)
        

Below, we are ading the tonic symbol. We load the already crawled MusicBrainz metadata to fetch the makam. Then, we fetch the tonic symbol by looking up in a makam tonic dictionary

In [3]:
tonic_dict = AhenkIdentifier._get_dict('tonic')  # load the makam tonic dict

for ii, (key, val) in enumerate(anno_dict.items()):
    # load the metadata
    meta_file = os.path.join('../metadata/%s.json' % key)
    meta = json.load(open(meta_file))
    
    try:
        makams = set(mm['attribute_key'] for mm in meta['makam'])

        tonics = set(AhenkIdentifier._get_tonic_symbol_from_makam(
                mm, tonic_dict)[0] for mm in makams)

        if len(tonics) == 1:
            for aa in anno_dict[key]['annotations']:
                if aa['tonic_symbol']:  # don't override
                    if aa['tonic_symbol'] != list(tonics)[0]:
                        print str(ii) + ' ' + key
                        print "... Tonic symbol mismatch: " + aa['tonic_symbol'] + ' -> ' + list(tonics)[0]
                else:
                    aa['tonic_symbol'] = list(tonics)[0]
        elif len(tonics) > 1:
            print str(ii) + ' ' + key
            print "... More than one tonic symbol for: " + ', '.join(makams)
        else:  # not tonic
            print str(ii) + ' ' + key
            print "... Tonic symbol not available for: " + ', '.join(makams)
    except KeyError:
        print str(ii) + ' ' + key
        print "... No makam info available"

161 754d6f70-e460-4e8b-89ba-cac1e1ebeacd
... Tonic symbol not available for: 
191 a8cd450e-e8d9-4245-b670-2a9626e14b0c
... Tonic symbol not available for: 
476 d2ebf684-8b56-418c-866a-0fa6b8acda1e
... More than one tonic symbol for: acemkurdi, mahur
622 00022b8f-a731-49dd-9fcf-6bda7b297545
... Tonic symbol not available for: 
623 f9150889-a3b4-4031-bee7-287a8e3da356
... Tonic symbol not available for: 
679 686ee26e-a03c-4823-8fc5-056c7aba1013
... Tonic symbol not available for: 
773 761ccd82-281b-4f54-95fe-1d4bb27e5585
... Tonic symbol not available for: 
924 06a7aef4-a5cc-41e6-b5bb-7a529fea17c0
... No makam info available
947 79f1ac75-1c45-4631-886a-1c309799ed21
... More than one tonic symbol for: kurdilihicazkar, muhayyerkurdi
1063 12c0f5c7-ca7e-4327-a819-7e3bb38454d9
... No makam info available
1212 0a49eb8e-79f5-429e-9846-a36dda35dd23
... Tonic symbol not available for: 
1316 0fa2576a-a189-4b73-b0fb-42ddbcffa64d
... Tonic symbol not available for: 
1418 78d58438-81fc-41a3-8ecf-30e6

In [4]:
json.dump(anno_dict, open('../annotations.json', 'w'), indent=2)